# CS 414 Codes - Part 2

## Simulation

In [3]:
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import Point
from itertools import combinations
from scipy.stats import pearsonr, spearmanr
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from haversine import haversine
from tqdm import tqdm
import warnings, re, os
from unidecode import unidecode
import copy

warnings.filterwarnings("ignore")

traf_net = nx.read_gexf("pos_prop_new.gexf")
nx.set_node_attributes(traf_net, 0, 'current_car_count')

In [ ]:
## Simulate with params
flow_speed = 5 # percentile that will exit the node in next iteration
time_span = 100 # number of hours to iterate

nx.set_node_attributes(traf_net, {"sxk9sr":10000}, 'current_car_count')

nx.write_gexf(traf_net, "diff_iter_0.gexf")

## Start sim
for i in range(time_span):
    print("Running for hour {}...".format(i+1))
    
    next_graph = nx.read_gexf("pos_prop_new.gexf")
    nx.set_node_attributes(next_graph, 0, 'current_car_count')
    
    change_dict = {}
    
    for node in traf_net.nodes(data="current_car_count"):
        
        if node[1] != 0:
            #print("Considering node {}".format(node[0]))
            out_amount_possible = node[1]*((flow_speed * np.log10(node[1]+1)) / 100.0)
            out_weight = [(to, traf_net[fr][to]["weight"]) for fr, to in traf_net.out_edges(node)]
            w_sum = sum([x[1] for x in out_weight])
            
            # update node and go next
            
            unit_update = out_amount_possible / w_sum * 1.0
            
            for item in out_weight:
                if item[0] not in change_dict.keys():
                    change_dict[item[0]] = (item[1] * unit_update)
                else:        
                    change_dict[item[0]] += (item[1] * unit_update)
                
                if node[0] not in change_dict.keys():
                    change_dict[node[0]] = -(item[1] * unit_update)
                else:        
                    change_dict[node[0]] -= (item[1] * unit_update)
            
    for node in next_graph.nodes(data="current_car_count"): 
        if node[0] in change_dict.keys() : 
            nx.set_node_attributes(next_graph, {node[0] : int(traf_net.nodes(data="current_car_count")[node[0]] + change_dict[node[0]])}, 'current_car_count')

    #print([node[1] for node in traf_net.nodes(data="current_car_count") if node[1] != 0])
    #print([node[1] for node in next_graph.nodes(data="current_car_count") if node[1] != 0])

    traf_net = copy.deepcopy(next_graph) # switch to next
    if (i+1) % 20 == 0 : nx.write_gexf(traf_net, "diff_iter_{}.gexf".format(i+1))
    next_graph.clear()

Running for hour 1...
Running for hour 2...
Running for hour 3...
Running for hour 4...
Running for hour 5...
Running for hour 6...
Running for hour 7...
Running for hour 8...
Running for hour 9...
Running for hour 10...
Running for hour 11...
Running for hour 12...
Running for hour 13...
Running for hour 14...
Running for hour 15...
Running for hour 16...
Running for hour 17...
Running for hour 18...
Running for hour 19...
Running for hour 20...
Running for hour 21...
Running for hour 22...
Running for hour 23...
Running for hour 24...
Running for hour 25...
Running for hour 26...
Running for hour 27...
Running for hour 28...
Running for hour 29...
Running for hour 30...
Running for hour 31...
Running for hour 32...
Running for hour 33...
Running for hour 34...
Running for hour 35...
Running for hour 36...
Running for hour 37...
Running for hour 38...
Running for hour 39...
Running for hour 40...
Running for hour 41...
Running for hour 42...
Running for hour 43...
Running for hour 44.

## Network Propagation

### Setup

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import Point
from itertools import combinations
from scipy.stats import pearsonr, spearmanr
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from haversine import haversine
from tqdm import tqdm
import warnings, re, os
from unidecode import unidecode
warnings.filterwarnings("ignore")

In [2]:
# ALL MONTHS
f_list = sorted([item for item in os.listdir(".") if re.search(r'.*csv$', item)])

df = pd.read_csv(f_list[0])

for i in tqdm(range(1, len(f_list))): df = pd.concat([df, pd.read_csv(f_list[i])], ignore_index=True, sort=False)

df.head()

100%|███████████████████████████████████████████| 35/35 [00:42<00:00,  1.22s/it]


,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,2020-01-01 00:00:00,28.811646,41.080627,sxk3xw,135,18,81,132
1,2020-01-01 00:00:00,29.108276,40.987244,sxk9nm,143,10,73,162
2,2020-01-01 00:00:00,29.097290,41.003723,sxk9q0,128,6,50,110
3,2020-01-01 00:00:00,28.679810,40.998230,sxk3hx,111,22,68,101
4,2020-01-01 00:00:00,28.020630,41.042175,sx7cmx,99,99,99,1


In [3]:
df.columns = ["DATE_TIME", "LONGITUDE", "LATITUDE", "GEOHASH", "MAXIMUM_SPEED", 
              "MINIMUM_SPEED", "AVERAGE_SPEED", "NUMBER_OF_VEHICLES"]
df["DATE_TIME"] = pd.to_datetime(df["DATE_TIME"])

df_t = df.set_index("DATE_TIME").groupby("GEOHASH").mean().reset_index()

df_t.head()

,GEOHASH,LONGITUDE,LATITUDE,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,sx5zu6,27.965698,40.751038,21.000000,21.000000,21.000000,1.000000
1,sx7chk,27.965698,40.981750,117.481279,58.836708,83.515438,14.468498
2,sx7chm,27.965698,40.987244,114.455996,31.805961,72.352423,24.616486
3,sx7chq,27.965698,40.992737,16.000000,2.000000,9.000000,5.000000
4,sx7chr,27.965698,40.998230,28.000000,6.000000,14.000000,11.000000


In [4]:
def get_dist(row1, row2):
    return  haversine((row1["LATITUDE"], row1["LONGITUDE"]), (row2["LATITUDE"], row2["LONGITUDE"])) 

def get_reg(point, gdf):
    for idx,row in gdf.iterrows():
        if row["geometry"].contains(point):
            return row["name_corrected"]
    return None
    
geo_df = df_t[["GEOHASH", "LATITUDE", "LONGITUDE"]].drop_duplicates(subset=['GEOHASH']).set_index("GEOHASH")

gdf = gp.read_file("istanbul-districts.geojson")
gdf["name_corrected"] = gdf["name"].apply(lambda x : unidecode(x))
geo_df["COORDS"] = [Point(row["LONGITUDE"],row["LATITUDE"]) for idx,row in geo_df.iterrows()]
geo_df["GROUP"] = geo_df["COORDS"].apply(lambda x : get_reg(x, gdf))
geo_df = geo_df.dropna(subset=['GROUP'])

dists = {}
combs =  list(combinations(geo_df.index, 2))
prop_network = nx.DiGraph()

for i in tqdm(range(len(combs))):
    comb = combs[i]
    if get_dist(geo_df.loc[comb[0]], geo_df.loc[comb[1]]) < 1:
        
        prop_network.add_node(comb[0], region = geo_df.loc[comb[0]]["GROUP"], lat = geo_df.loc[comb[0]]["LATITUDE"], lon = geo_df.loc[comb[0]]["LONGITUDE"])
        prop_network.add_node(comb[1], region = geo_df.loc[comb[1]]["GROUP"], lat = geo_df.loc[comb[1]]["LATITUDE"], lon = geo_df.loc[comb[1]]["LONGITUDE"])
        
        prop_network.add_edge(comb[0], comb[1], weight = 0)
        prop_network.add_edge(comb[1], comb[0], weight = 0)

print(prop_network)
geo_df.head()

100%|██████████████████████████████| 9867903/9867903 [07:42<00:00, 21321.31it/s]

DiGraph with 4376 nodes and 13588 edges


,LATITUDE,LONGITUDE,COORDS,GROUP
GEOHASH,,,,
sx7cmp,41.042175,27.998657,POINT (27.9986572265625 41.0421752929688),Silivri
sx7cmq,41.036682,28.009644,POINT (28.0096435546875 41.03668212890629),Silivri
sx7cmr,41.042175,28.009644,POINT (28.0096435546875 41.0421752929688),Silivri
sx7cmx,41.042175,28.020630,POINT (28.0206298828125 41.0421752929688),Silivri
sx7cmz,41.042175,28.031616,POINT (28.0316162109375 41.04217529296879),Silivri


In [5]:
time_series = np.unique(df["DATE_TIME"])
alpha = 0.0001

def update_weight(w, d_true, d_pred, alpha=alpha):
    # Linear propagation case (signs of changes are the same)
    
    w_new = 0
    
    if (d_true >= 0 and d_pred >= 0) or (d_true <= 0 and d_pred <= 0):
        w_new = w - alpha * (abs(d_pred) - abs(d_true)) # Adjust the confidence from predictions
    
    # No simple association
    else:
        w_new =  w - alpha * (abs(d_pred) + abs(d_true)) # Reduce weight / confidence
    
    # If no association, don't reduce further
    if w_new < 0 or np.isnan(w_new):
        return 0
    else:
        return w_new
    
def grad_desc(edge, cur_df, prev_df, feature, prop_network=prop_network, alpha=alpha):
        
    # Grab initial weight
    w_0, w_1 = prop_network[edge[0]][edge[1]]["weight"], prop_network[edge[1]][edge[0]]["weight"]
    
    try:

        # Calculate true diff
        d_0_true, d_1_true = cur_df[feature].loc[edge[0]] - prev_df[feature].loc[edge[0]], cur_df[feature].loc[edge[1]] - prev_df[feature].loc[edge[1]]

        # 0 assoc 1, 1 assoc 0
        d_0_pred, d_1_pred = d_1_true * w_0, d_0_true * w_1

        # Update weight
        return (update_weight(w_0, d_0_true, d_0_pred), update_weight(w_1, d_1_true, d_1_pred))
    
    except KeyError: # Feature / time not recorded for given pair  
        return w_0, w_1
    
for i in tqdm(range(1, len(time_series))):
    
    cur_df, prev_df = df[df["DATE_TIME"] == time_series[i]].set_index("GEOHASH"), df[df["DATE_TIME"] == time_series[i-1]].set_index("GEOHASH")
    
    for edge in prop_network.edges(data=True):
        
        prop_network[edge[0]][edge[1]]["weight"], prop_network[edge[1]][edge[0]]["weight"] = grad_desc(edge, cur_df, prev_df, feature = "NUMBER_OF_VEHICLES")
    

100%|███████████████████████████████████| 25713/25713 [9:57:08<00:00,  1.39s/it]


In [ ]:
weights = [prop_network[edge[0]][edge[1]]["weight"] for edge in prop_network.edges]
print("Max:{}\nMin:{}".format(max(weights), min(weights)))

weights

In [7]:
nx.write_gexf(prop_network, "pos_prop_new.gexf")

In [24]:
df.head()

,GEOHASH,LONGITUDE,LATITUDE,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,sx5zu6,27.965698,40.751038,21.000000,21.000000,21.000000,1.000000
1,sx7chk,27.965698,40.981750,117.481279,58.836708,83.515438,14.468498
2,sx7chm,27.965698,40.987244,114.455996,31.805961,72.352423,24.616486
3,sx7chq,27.965698,40.992737,16.000000,2.000000,9.000000,5.000000
4,sx7chr,27.965698,40.998230,28.000000,6.000000,14.000000,11.000000
